In [1]:
# Importing libraries
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

# MERGE ALL THE DATA 


In [2]:
# Importing datasets
filepaths_left={
    "pvs1_gps_mpu" : r"archive/PVS 1/dataset_gps_mpu_left.csv",
    "pvs2_gps_mpu" : r"archive/PVS 2/dataset_gps_mpu_left.csv",
    "pvs3_gps_mpu" : r"archive/PVS 3/dataset_gps_mpu_left.csv",
    "pvs4_gps_mpu" : r"archive/PVS 4/dataset_gps_mpu_left.csv",
    "pvs5_gps_mpu" : r"archive/PVS 5/dataset_gps_mpu_left.csv",
    "pvs6_gps_mpu" : r"archive/PVS 6/dataset_gps_mpu_left.csv",
    "pvs7_gps_mpu" : r"archive/PVS 7/dataset_gps_mpu_left.csv",
    "pvs8_gps_mpu" : r"archive/PVS 8/dataset_gps_mpu_left.csv",
    "pvs9_gps_mpu" : r"archive/PVS 9/dataset_gps_mpu_left.csv"
}

filepaths_right={
    "pvs1_gps_mpu" : r"archive/PVS 1/dataset_gps_mpu_right.csv",
    "pvs2_gps_mpu" : r"archive/PVS 2/dataset_gps_mpu_right.csv",
    "pvs3_gps_mpu" : r"archive/PVS 3/dataset_gps_mpu_right.csv",
    "pvs4_gps_mpu" : r"archive/PVS 4/dataset_gps_mpu_right.csv",
    "pvs5_gps_mpu" : r"archive/PVS 5/dataset_gps_mpu_right.csv",
    "pvs6_gps_mpu" : r"archive/PVS 6/dataset_gps_mpu_right.csv",
    "pvs7_gps_mpu" : r"archive/PVS 7/dataset_gps_mpu_right.csv",
    "pvs8_gps_mpu" : r"archive/PVS 8/dataset_gps_mpu_right.csv",
    "pvs9_gps_mpu" : r"archive/PVS 9/dataset_gps_mpu_right.csv"


}

# Left and right corresponds to sensordata from either side of the cars
datasets_left = {name:pd.read_csv(path) for name,path in filepaths_left.items()}
datasets_right = {name:pd.read_csv(path) for name,path in filepaths_right.items()}

In [6]:
def merge(left, right, name ):
    pvs= pd.concat([left, right], axis=1)
    length= len(left)

    # Renaming labels
    pvs.columns = [
    'timestamp', 'acc_x_dashboard_l', 'acc_y_dashboard_l', 'acc_z_dashboard_l',
    'acc_x_above_suspension_l', 'acc_y_above_suspension_l', 'acc_z_above_suspension_l', 
    'acc_x_below_suspension_l', 'acc_y_below_suspension_l', 'acc_z_below_suspension_l', 
    'gyro_x_dashboard_l', 'gyro_y_dashboard_l', 'gyro_z_dashboard_l', 'gyro_x_above_suspension_l', 
    'gyro_y_above_suspension_l', 'gyro_z_above_suspension_l', 'gyro_x_below_suspension_l', 
    'gyro_y_below_suspension_l', 'gyro_z_below_suspension_l', 'mag_x_dashboard_l', 'mag_y_dashboard_l', 
    'mag_z_dashboard_l', 'mag_x_above_suspension_l', 'mag_y_above_suspension_l', 'mag_z_above_suspension_l', 
    'temp_dashboard_l', 'temp_above_suspension_l', 'temp_below_suspension_l', 'timestamp_gps', 
    'latitude', 'longitude', 'speed', 'timestamp', 'acc_x_dashboard_r', 'acc_y_dashboard_r', 
    'acc_z_dashboard_r', 'acc_x_above_suspension_r', 'acc_y_above_suspension_r', 
    'acc_z_above_suspension_r', 'acc_x_below_suspension_r', 'acc_y_below_suspension_r', 
    'acc_z_below_suspension_r', 'gyro_x_dashboard_r', 'gyro_y_dashboard_r', 'gyro_z_dashboard_r', 
    'gyro_x_above_suspension_r', 'gyro_y_above_suspension_r', 'gyro_z_above_suspension_r', 
    'gyro_x_below_suspension_r', 'gyro_y_below_suspension_r', 'gyro_z_below_suspension_r', 
    'mag_x_dashboard_r', 'mag_y_dashboard_r', 'mag_z_dashboard_r', 'mag_x_above_suspension_r', 
    'mag_y_above_suspension_r', 'mag_z_above_suspension_r', 'temp_dashboard_r', 'temp_above_suspension_r', 
    'temp_below_suspension_r', 'timestamp_gps', 'latitude', 'longitude', 'speed'
    ]
    

    # Remove duplicate columns (those with the same name, e.g., 'timestamp_gps', 'latitude', etc.)
    pvs_removed = pvs.loc[:, ~pvs.columns.duplicated()]

    # Columns to keep. We try these first
    to_keep=["timestamp",
             "acc_x_dashboard_l",
             "acc_y_dashboard_l",
             "acc_z_dashboard_l",
             "speed",
             "gyro_x_dashboard_l",
             "gyro_y_dashboard_l",
             "gyro_z_dashboard_l"
            ]
    
    pvs_removed=pvs_removed[to_keep]

    #create the driver column ( target )
    if name in ("pvs1_gps_mpu","pvs4_gps_mpu","pvs7_gps_mpu"):
        pvs_removed['Driver'] = 1
    elif name in ("pvs2_gps_mpu","pvs5_gps_mpu","pvs8_gps_mpu"):
        pvs_removed['Driver'] = 2
    else: 
        pvs_removed['Driver'] = 3
        
    return pvs_removed

### CREATE TARGETS
We want to predict the driver, and we know that each PVS contains data regarding one single driver. 
As a conseguence, we can simply create a dataframe containing the same values repeated n times for each PVS.

The datasets refer to the following drivers: 

PVS1--> 1

PVS2--> 2

PVS3--> 3

PVS4--> 1

PVS5--> 2

PVS6--> 3

PVS7--> 1

PVS8--> 2

PVS9--> 3


In [7]:
# Using the merge function to create the new versions of the datasets
datasets = {name: merge(datasets_left[name],datasets_right[name], name) for name in datasets_right.keys()}

In [18]:
#print all the features 
columns=set()
for name,dataset in datasets.items():
    columns.update(list(dataset.columns))

for el in columns:
    print(el)

Driver
gyro_z_dashboard_l
gyro_x_dashboard_l
acc_z_dashboard_l
acc_x_dashboard_l
gyro_y_dashboard_l
speed
timestamp
acc_y_dashboard_l


In [8]:
datasets["pvs1_gps_mpu"].head()

,timestamp,acc_x_dashboard_l,acc_y_dashboard_l,acc_z_dashboard_l,speed,gyro_x_dashboard_l,gyro_y_dashboard_l,gyro_z_dashboard_l
0,1.577219e+09,0.365116,0.167893,9.793961,0.009128,-0.133896,-0.018883,0.138092
1,1.577219e+09,0.392649,0.176273,9.771216,0.009128,-0.027084,-0.003624,0.000763
2,1.577219e+09,0.409408,0.181062,9.732909,0.009128,0.125504,-0.186729,-0.090790
3,1.577219e+09,0.371101,0.164302,9.749668,0.009128,-0.088120,-0.034142,0.046539
4,1.577219e+09,0.390255,0.159514,9.869378,0.009128,-0.179672,0.118446,-0.182343


# WORK ON CORRELATION (OPTIONAL SO FAR)

In [ ]:
def compute_correlation(df,plot=False):

    # Only compute using the numeric data types in the dataset
    numeric_df = df.select_dtypes(include=['number'])
    
    #print(numeric_df.columns)
    correlation_matrix=numeric_df.corr()

    if not plot:
        return correlation_matrix
        
    # Create the heatmap
    plt.figure(figsize=(18, 14))  # Increase the size of the figure
    
    # Plot the heatmap with adjusted figure size
    sns.heatmap(correlation_matrix, annot=False, fmt='.2f', cmap='coolwarm', linewidths=0.5,vmin=-1, vmax=1)
    
    # Rotate the tick labels for better readability and shrink font size
    plt.xticks(rotation=90, fontsize=8)  # Rotate x-axis labels by 90 degrees and adjust font size
    plt.yticks(rotation=0, fontsize=8)   # Keep y-axis labels horizontal and adjust font size
    
    # Add a title with smaller font size
    plt.title("Correlation Matrix", fontsize=12)
    
    # Show the plot with a tight layout
    plt.tight_layout()  # Adjust layout to prevent clipping
    plt.show()

    return correlation_matrix

In [22]:
datasets_correlation={name: compute_correlation(datasets[name]) for name in datasets.keys()}

## REDUCE THE DATA
Try to reduce the data by merging k consecutive lines ( reduce by a factor of k )

In [11]:
#reduce the number of rows of the dataset
def reduce_df(df,ratio):
    new = pd.DataFrame()
    for i in range(0, len(df) - 1, ratio):  # Step by 2 to get pairs

        if i%10000==0:
            print(i)
            
        row1 = df.iloc[i]
        row2 = df.iloc[i + 1]
        # Calculate the mean of the pair (assuming numeric data)
        merged_row = (row1 + row2) / 2
        
        # Append the result as a new row to `new`
        new = pd.concat([new, merged_row.to_frame().T], ignore_index=True)
        
    return new

ratio=4
datasets_reduced={name:reduce_df(datasets[name],ratio) for name in datasets.keys()}




0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
0
10000
20000
30000
40000
50000
60000
70000
80000
90000


## WORK ON A WINDOW

In [12]:
def delete_timestamp(df):
    to_remove=["timestamp_max","timestamp_min","timestamp_mean","timestamp_STD"]
    new= df.drop(columns=to_remove, axis=1, errors='ignore')

    return new

def update_name(df):
    #print(df.columns)
    #df = df.loc[:, ~df.columns.str.contains('timestamp')]  # Remove columns containing 'timestamp'
    new_column_names = [ 'timestamp_max','acc_x_dashboard_l_max', 'acc_y_dashboard_l_max', 
        'acc_z_dashboard_l_max', 'speed_max', 'gyro_x_dashboard_l_max', 
        'gyro_y_dashboard_l_max', 'gyro_z_dashboard_l_max', 
        'timestamp_min', 'acc_x_dashboard_l_min', 'acc_y_dashboard_l_min', 
        'acc_z_dashboard_l_min', 'speed_min', 'gyro_x_dashboard_l_min', 
        'gyro_y_dashboard_l_min', 'gyro_z_dashboard_l_min', 
        'timestamp_mean', 'acc_x_dashboard_l_mean', 'acc_y_dashboard_l_mean', 
        'acc_z_dashboard_l_mean', 'speed_mean', 'gyro_x_dashboard_l_mean', 
        'gyro_y_dashboard_l_mean', 'gyro_z_dashboard_l_mean', 
        'timestamp_STD_', 'acc_x_dashboard_l_STD_', 'acc_y_dashboard_l_STD_', 
        'acc_z_dashboard_l_STD_', 'speed_STD_', 'gyro_x_dashboard_l_STD_', 
        'gyro_y_dashboard_l_STD_', 'gyro_z_dashboard_l_STD_',
         'acc_x_dashboard_l_jerk', 'acc_y_dashboard_l_jerk', 
        'acc_z_dashboard_l_jerk', 'speed_jerk', 'gyro_x_dashboard_l_jerk', 
        'gyro_y_dashboard_l_jerk', 'gyro_z_dashboard_l_jerk'
    ]
    df.columns = new_column_names
    return df



#let's define a window
def computeWindow(name,df, windowSize=100):
    print("--------------------------")
    print(f"computing dataset {name}")
    #define empty dataframe  
    windowed_df=pd.DataFrame()

    #define starting and ending index
    for start_idx in range(len(df)-windowSize+1):
        end_idx=start_idx+windowSize

        #extract rows belonging to the window
        window=df.iloc[start_idx:end_idx]

        #take beginning timestamp and ending timestamp
        start_timestamp=window.iloc[0,0]
        end_timestamp=window.iloc[-1,0]

        
        #keep only sensor data
        sensor_data_window=window.iloc[:,1:]

        #compute metrics for the specific window
        max_values=window.max()
        min_values=window.min()
        mean_values=window.mean()
        std_values=window.std()
        jerk_values=(sensor_data_window.iloc[-1]-sensor_data_window.iloc[0])/(end_timestamp-start_timestamp)

        #concate them (place side by side)
        new_row=pd.concat([max_values,min_values,mean_values,std_values,jerk_values])
        new_row = new_row.to_frame().T 

        windowed_df = pd.concat([windowed_df, new_row], ignore_index=True)

        if start_idx%10000==0:
            print(start_idx)
            
            
    return delete_timestamp(update_name(windowed_df))


In [13]:
#winodew the datasets
datasets_windowed={name:computeWindow(name,datasets_reduced[name]) for name in datasets.keys()}


--------------------------
computing dataset pvs1_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs2_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs3_gps_mpu
0
10000
20000
--------------------------
computing dataset pvs4_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs5_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs6_gps_mpu
0
10000
20000
--------------------------
computing dataset pvs7_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs8_gps_mpu
0
10000
20000
30000
--------------------------
computing dataset pvs9_gps_mpu
0
10000
20000


In [138]:
datasets_windowed["pvs3_gps_mpu"].head()

,timestamp_max,acc_x_dashboard_l_max,acc_y_dashboard_l_max,acc_z_dashboard_l_max,speed_max,gyro_x_dashboard_l_max,gyro_y_dashboard_l_max,gyro_z_dashboard_l_max,timestamp_min,acc_x_dashboard_l_min,...,gyro_x_dashboard_l_STD_,gyro_y_dashboard_l_STD_,gyro_z_dashboard_l_STD_,acc_x_dashboard_l_jerk,acc_y_dashboard_l_jerk,acc_z_dashboard_l_jerk,speed_jerk,gyro_x_dashboard_l_jerk,gyro_y_dashboard_l_jerk,gyro_z_dashboard_l_jerk
0,1.577223e+09,0.021308,0.037529,9.891764,0.009432,0.304222,0.173187,0.193024,1.577223e+09,-0.065481,...,0.091692,0.088113,0.093786,-0.000302,-0.000302,0.017836,0.000246,-0.015413,0.001927,-0.003853
1,1.577223e+09,0.021308,0.037529,9.891764,0.009432,0.304222,0.173187,0.193024,1.577223e+09,-0.065481,...,0.092079,0.088097,0.093472,0.004686,-0.001058,-0.001663,0.000246,-0.036606,-0.052019,0.001927
2,1.577223e+09,0.021308,0.037529,9.891764,0.009432,0.304222,0.119781,0.193024,1.577223e+09,-0.065481,...,0.091987,0.086446,0.093455,-0.000756,-0.004232,-0.002570,0.000246,-0.055872,-0.059725,-0.009633
3,1.577223e+09,0.021308,0.037529,9.891764,0.009432,0.304222,0.119781,0.193024,1.577223e+09,-0.065481,...,0.089988,0.085906,0.093497,-0.006348,-0.014208,0.006651,0.000246,-0.011560,-0.046239,-0.015413
4,1.577223e+09,0.021308,0.037529,9.891764,0.009432,0.304222,0.119781,0.193024,1.577223e+09,-0.065481,...,0.090007,0.084997,0.094386,0.003628,-0.000605,-0.000605,0.000246,0.023119,-0.023119,-0.007706


In [ ]:
#CONCATENATE ALL OF THEM AND FIND THE FINAL DATASET 
combined_df = pd.concat(datasets_windowed_final.values(), axis=0, ignore_index=True)

combined_df.head()
#combined_df.size()
print(type(combined_df))
print(combined_df.shape)



(269336, 36)


## WORK AGAIN ON CORRELATION (NOW INTERESTING)

In [15]:
corr_matrix_final=compute_correlation(combined_df,False)

In [18]:
def remove_highly_correlated(df,corr,threshold=0.9):
    #take absoulute values
    corr_matrix_abs=corr.abs()
    
    #take only upper diagonal 
    mask = np.triu(np.ones_like(corr, dtype=bool), k=1)
    
    highly_correlated_features=np.where(corr_matrix_abs*mask>threshold)
    
    #print and store highly correlated features
    
    to_drop=set()
    for i,j in zip(*highly_correlated_features):
        #print( f"highly correlated pair: {corr_matrix_abs.columns[i]}-{corr_matrix_abs.columns[j]},(correlation: {corr_matrix_abs.iloc[i, j]:.2f})")
        to_drop.add(corr_matrix_abs.columns[j])
        #print(corr_matrix_abs.columns[j])
    
    #remove to_drop columns
    cleaned_df=df.drop(columns=to_drop)
    print(f"droopped columns {to_drop}")
    #print(f"removed {count} columns")
    print(f"new dataframe has shape {cleaned_df.shape}")

    return cleaned_df,to_drop

In [30]:
cleaned_final_df, dropped_columns = remove_highly_correlated(combined_df, corr_matrix_final, 0.6)

final_metrics_correlation = {name: compute_correlation(cleaned_final_df[name]) for name in cleaned_final_df.columns}


droopped columns {'speed_min', 'gyro_y_dashboard_l_max', 'gyro_y_dashboard_l_STD_', 'acc_z_dashboard_l_STD_', 'gyro_x_dashboard_l_max', 'gyro_z_dashboard_l_mean', 'acc_z_dashboard_l_max', 'acc_x_dashboard_l_STD_', 'acc_y_dashboard_l_STD_', 'acc_y_dashboard_l_max', 'acc_x_dashboard_l_min', 'gyro_x_dashboard_l_min', 'speed_mean', 'acc_z_dashboard_l_min', 'acc_y_dashboard_l_min', 'gyro_y_dashboard_l_min', 'gyro_x_dashboard_l_STD_'}
new dataframe has shape (269336, 19)
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "c:\Users\bruker\venv\lib\site-packages\IPython\core\interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\bruker\AppData\Local\Temp\ipykernel_38728\2447120228.py", line 3, in <module>
    final_metrics_correlation = {name: compute_correlation(cleaned_final_df[name]) for name in cleaned_final_df.columns}
  File "C:\Users\bruker\AppData\Local\Temp\ipykernel_38728\2447120228.py", line 3, in <dictcomp>
    final_metrics_correlation = {name: compute_correlation(cleaned_final_df[name]) for name in cleaned_final_df.columns}
  File "C:\Users\bruker\AppData\Local\Temp\ipykernel_38728\3691109450.py", line 2, in compute_correlation
    numeric_df = df.select_dtypes(include=['number'])
  File "c:\Users\bruker\venv\lib\site-packages\pandas\core\generic.py", line 5575, in __getattr__
    return object.__getattribute__(self, name)
AttributeError: 'Series' object has no attribute 'select_dtypes'

D